In [2]:
from __future__ import print_function

import pickle
import seaborn as sns
import numpy as np
import pandas as pd
import json
import os
import copy
import time
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import sys
import time
import utils

In [3]:
print ("Started!!")

st = time.time()
symbols = utils.get_sap_symbols('sap500')
np.random.shuffle(symbols)
chosen_symbols = symbols[:10]
start_date="2006-10-01"
end_date="2016-10-01"
# use Open data
input_data = utils.get_data_list_key(chosen_symbols, start_date, end_date)
elapsed = time.time() - st
print ("time for getting data:", elapsed)

Started!!
we cound not fetch data from the following companies
['VRSK']
time for getting data: 21.6230559349


In [4]:
train_st = pd.Timestamp("2006-10-01")
train_end = pd.Timestamp("2015-10-01")
test_st = pd.Timestamp("2015-10-02")
test_end = pd.Timestamp("2016-10-01")

train_data = input_data.loc[(input_data.index >= train_st) & (input_data.index <= train_end)]
test_data = input_data.loc[(input_data.index >= test_st) & (input_data.index <= test_end)]

# DDPG for trading


Given data, you are going to learn how to manage your portfolio.

In [5]:
# import numpy as np 
# input_data = np.zeros((505, 99))
# n_stock = len(input_tilde.values[0])
n_stock = 10

class MultiDDPGConfig(object):
    activation = 'relu'
    gamma = 0.99
    history_length = 6
    n_stock = n_stock
    n_smooth = 5
    n_down = 5
    k_w = 3
    n_hidden = 100
    n_batch = 32
    n_epochs = 10
    n_feature = 32
    update_rate = 1e-1
    learning_rate = 1e-2

index = pd.date_range('23/10/2016', periods=100, freq='D')
value = np.random.normal(0, 1, (len(index), n_stock))
input_data = pd.DataFrame(value, index=index)

In [6]:
from collections import deque, namedtuple
import numpy as np
from numpy.random import choice

Experiecne = namedtuple('Experience', 'state0,  action, reward, state1')

class RingBuffer(object):
    def __init__(self, maxlen):
        self.maxlen = maxlen
        self.start = 0
        self.length = 0
        # self.data = [None for _ in range(maxlen)]
        self.data = []
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        if idx< 0 or idx >= self.length:
            raise KeyError()
        return self.data[(self.start + idx) % self.maxlen]
    
    def append(self, v):
        if self.length < self.maxlen:
            # We have space, simply increase the length
            self.length += 1
        elif self.length == self.maxlen:
            # No space, "remove" the first item
            self.data[:-1] = self.data[1:]
        else:
            # This should never happen
            raise RuntimeError()
        self.data.append(v)
        
class SequentialMemory(object):
    def __init__(self, limit=1000):
        self.limit = limit
        self.priority = []
        self.actions = RingBuffer(limit)
        self.rewards = RingBuffer(limit)
        self.observations = RingBuffer(limit)
        self.batch_idx = None
        # print('priority:', self.priority)

        
    def sample(self, batch_size, window_length, alpha=1.0, beta=1.0, epsilon=0.05):
        # udpate priority when sampling
        if len(self.priority) > self.limit:
            self.priority = self.priority[-self.limit:]
        # draw random indexes such that is bigger than window_length to enough length data
        index_space = np.arange(window_length, self.nb_entries)
        # prioritized sample
        p = np.array(self.priority)[window_length:]
        p_tilde = p + np.ones(self.nb_entries - window_length) * np.mean(p) * epsilon
        p_tilde[-1] = np.mean(p)
        p_tilde = p_tilde ** alpha
        p_tilde = p_tilde / np.sum(p_tilde)
        batch_idx = choice(index_space, p=p_tilde, size=batch_size - 1)
        # take the newest data
        batch_idx = np.concatenate((batch_idx, [self.nb_entries - 1]))
        assert len(batch_idx) == batch_size
        # keep batch_idx to update pritority
        self.batch_idx = batch_idx
        
        # weights to modify biased update
        weights = 1. / (p_tilde**beta)
        weights = weights / np.max(weights)
        ret_w = weights[batch_idx - window_length]
        
        # create experiences
        state0 = np.array([[self.observations[i] for i in range(idx - window_length,idx)] for idx in batch_idx])
        action = np.array([self.actions[idx - 1] for idx in batch_idx])
        reward = np.array([self.rewards[idx - 1] for idx in batch_idx])
        state1 = np.array([[self.observations[i] for i in range(idx - window_length + 1,idx + 1)] for idx in batch_idx])
        return Experiecne(state0, action, reward, state1), ret_w
    
    def sample_state(self, batch_size, window_length, alpha=0.5, epsilon=0.05):
        # udpate priority when sampling
        if len(self.priority) > self.limit:
            self.priority = self.priority[-self.limit:]
        # draw random indexes such that is bigger than window_length to enough length data
        index_space = np.arange(window_length, self.nb_entries)
        # prioritized sample
        p = np.array(self.priority)[window_length:]
        p_tilde = p + np.ones(self.nb_entries - window_length) * np.mean(p) * epsilon
        p_tilde[-1] = np.mean(p)
        p_tilde = p_tilde ** alpha
        p_tilde = p_tilde / np.sum(p_tilde)
        batch_idx = choice(index_space, p=p_tilde, size=batch_size - 1)
        # take the newest data
        batch_idx = np.concatenate((batch_idx, [self.nb_entries]))
        assert len(batch_idx) == batch_size
        
        # create experiences
        state = np.array([[self.observations[i] for i in range(idx - window_length,idx)] for idx in batch_idx])
        return state
    
    def sample_state_uniform(self, batch_size, window_length):
        # draw random indexes such that is bigger than window_length to enough length data
        batch_idx = np.random.random_integers(window_length, self.nb_entries - 1, size=batch_size - 1)
        # take the newest data
        batch_idx = np.concatenate((batch_idx, [self.nb_entries]))
        assert len(batch_idx) == batch_size
        
        # create experiences
        state = np.array([[self.observations[i] for i in range(idx - window_length, idx)] for idx in batch_idx])
        return state
    
    def update_priority(self,error):
        for idx, i in enumerate(self.batch_idx):
            self.priority[i] = error[idx]
    
    
    def append(self, observation, action, reward):
        self.observations.append(observation)
        self.actions.append(action)
        self.rewards.append(reward)
        # initialize new sample with 1
        self.priority.append(1.0)
    
    @property
    def nb_entries(self):
        return  len(self.observations)

In [13]:
import tensorflow as tf
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D 
from keras.layers.core import Flatten, Lambda
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers.core import Dense
from keras.engine.topology import Merge
from keras.layers.advanced_activations import PReLU
from keras.layers import SpatialDropout2D
from keras.layers import Dropout, Reshape
from keras import backend as K
import numpy as np
import pandas as pd
import time
# local library
from memory import SequentialMemory

class DQN(object):
    """Deep Deterministic Poilicy Gradient
    
    Basend on DDPG and Multiscale CNN, seek out 
    optimal strategy for stock trading.
    
    Available function
    - build_model: build network based on tensorflow and keras
    - train: given DateFrame stock data, train network
    - predict_action: givne DataFrame stock data, return optimal protfolio
    """
    
    def __init__(self, config):
        """initialized approximate value function
        
        config should have the following attributes
        
        Args:
            device: the device to use computation, e.g. '/gpu:0'
            gamma(float): the decay rate for value at RL
            history_length(int): input_length for each scale at CNN
            n_feature(int): the number of type of input 
                (e.g. the number of company to use at stock trading)
            trade_stock_idx(int): trading stock index
            gam (float): discount factor
            n_history(int): the nubmer of history that will be used as input
            n_smooth, n_down(int): the number of smoothed and down sampling input at CNN
            k_w(int): the size of filter at CNN
            n_hidden(int): the size of fully connected layer
            n_batch(int): the size of mini batch
            n_epochs(int): the training epoch for each time
            update_rate (0, 1): parameter for soft update
            learning_rate(float): learning rate for SGD
            memory_length(int): the length of Replay Memory
            n_memory(int): the number of different Replay Memories
            alpha, beta: [0, 1] parameters for Prioritized Replay Memories
            action_scale(float): the scale of initialized ation
        """
        self.device = config.device
        self.save_path = config.save_path
        self.is_load = config.is_load
        self.gamma = config.gamma
        self.history_length = config.history_length
        self.n_stock = config.n_stock
        self.n_feature = config.n_feature
        self.n_smooth = config.n_smooth
        self.n_down = config.n_down
        self.k_w = config.k_w
        self.n_hidden = config.n_hidden
        self.n_batch = config.n_batch
        self.n_epochs = config.n_epochs
        self.update_rate = config.update_rate
        self.alpha = config.alpha
        self.beta = config.beta
        self.lr = config.learning_rate
        self.memory_length = config.memory_length
        self.n_memory = config.n_memory
        self.action_scale = config.action_scale
        # the length of the data as input
        self.n_history = max(self.n_smooth + self.history_length, (self.n_down + 1) * self.history_length)
        print ("building model....")
        # have compatibility with new tensorflow
        tf.python.control_flow_ops = tf
        # avoid creating _LEARNING_PHASE outside the network
        K.clear_session()
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
        K.set_session(self.sess)
        with self.sess.as_default():
            with tf.device(self.device):
                self.build_model()
        print('finished building model!')
    
    def train(self, input_data, noise_scale=0.1):
        """training DDPG, where action is confined to integer space
        
        Args:
            data (DataFrame): stock price for self.n_feature companies
        """
        stock_data = input_data.values
        date = input_data.index
        T = len(stock_data)
        self.noise_scale = noise_scale
        
        # frequency for output
        print_freq = int(T / 100)
        if print_freq == 0:
            print_freq = 1
        print ("training....")
        st = time.time()
        # prioritizomg parameter
        db = (1 - self.beta) / 1000
        
        # result for return value
        values = [[] for _ in range(self.n_stock)]
        date_label = [[] for _ in range(self.n_stock)]
        date_use = []
        stock_use = []
        # keep half an year data 
        t0 = self.n_history + self.n_batch
        self.initialize_memory(stock_data[:t0], scale=noise_scale)
        plot_freq = 10
        save_freq = 10
        count = 0
        input_data.to_csv("stock_price.csv")
        for t in range(t0, T):
            stock_use.append(stock_data[t])
            date_use.append(date[t])
            action = self.predict_action(stock_data[t])
            # print(self.memory[0].observations.data)
            for i in range(self.n_stock):
                if action[i] == 0:
                    date_label[i].append(date[t])
                    values[i].append(stock_data[t][i])
            self.update_memory(stock_data[t])
            count += 1
            for epoch in range(self.n_epochs):    
                # select transition from pool
                self.update_weight()
                # update prioritizing paramter untill it goes over 1
            self.beta  += db
            if self.beta >= 1.0:
                self.beta = 1.0
            idx = np.random.randint(0, self.n_memory)
            
            experiences, weights = self.memory[idx].sample(self.n_batch, self.n_history, self.alpha, self.beta)
            max_idx = self.get_max_idx(experiences.state1)
            target_value = self.sess.run(self.target_value,
                                     feed_dict={self.state_target: experiences.state1,
                                 self.reward: experiences.reward,
                                               self.max_idx_target: max_idx})
            
            if t % print_freq == 0:
                print ("time:",  date[t])
                error = self.sess.run(self.error,
                              feed_dict={self.state: experiences.state0,
                                         self.target: target_value,
                                         self.reward: experiences.reward,
                                         K.learning_phase(): 0})
                print("error:", np.mean(error))
                action = self.predict_action(stock_data[t])
                print("portfolio:", action)
                print ("elapsed time", time.time() - st)
                print("********************************************************************")
                
            if count % plot_freq == 0:
                for i in range(self.n_stock):
                    result = pd.DataFrame(values[i], index=pd.DatetimeIndex(date_label[i]))
                    result.to_csv("exit_result_{}.csv".format(i))
                data_use = pd.DataFrame(stock_use, index=pd.DatetimeIndex(date_use))
                data_use.to_csv("stock_price.csv")
                
            if count % save_freq == 0:
                save_path = self.saver.save(self.sess, self.save_path)
                print("Model saved in file: %s" % self.save_path)

        save_path = self.saver.save(self.sess, self.save_path)
        print("Model saved in file: %s" % self.save_path)
        print ("finished training")
        
        return [pd.DataFrame(values[i], index=pd.DatetimeIndex(date_label[i])) for i in range(self.n_stock)]
    
    def predict_action(self, state):
        """Preduct Optimal Portfolio
        
        Args:
            state(float): stock data with size: [self.n_stock, ]
        Retrun:
            np.array with size: [self.n_stock, ]
        """
        pred_state = self.memory[0].sample_state_uniform(self.n_batch, self.n_history)
        new_state = pred_state[-1]
        new_state = np.concatenate((new_state[1:], [state]), axis=0)
        pred_state = np.concatenate((pred_state[:-1], [new_state]), axis=0)
        action = self.max_action.eval(
            session=self.sess,
            feed_dict={self.state: pred_state, K.learning_phase(): 0})[-1]
        # action = self.norm_action(action)
        return action
    
    def update_weight(self):
        # pararel memory update
        idx = np.random.randint(0, self.n_memory)
        experiences, weights = self.memory[idx].sample(self.n_batch, self.n_history, self.alpha, self.beta)
        max_idx = self.get_max_idx(experiences.state1)
        
        target_value = self.sess.run(self.target_value,
                                     feed_dict={self.state_target: experiences.state1,
                                 self.reward: experiences.reward,
                                               self.max_idx_target: max_idx})

        self.sess.run(self.critic_optim, 
                      feed_dict={self.state: experiences.state0,
                                 self.target: target_value,
                                 self.weights: weights,
                                 self.learning_rate: self.lr,
                                 K.learning_phase(): 1})  

        error = self.sess.run(self.error,
                              feed_dict={self.state: experiences.state0,
                                         self.target: target_value,
                                         self.reward: experiences.reward,
                                         K.learning_phase(): 0})
        self.memory[idx].update_priority(error)
        
                    
        # softupdate for critic network
        old_weights = self.critic_target.get_weights()
        new_weights = self.critic.get_weights()
        weights = [self.update_rate * new_w + (1 - self.update_rate) * old_w
                   for new_w, old_w in zip(new_weights, old_weights)]
        self.critic_target.set_weights(weights)
        
    def initialize_memory(self, stocks, scale=10):
        self.memory = []
        for i in range(self.n_memory):
            self.memory.append(SequentialMemory(self.memory_length))
        for t in range(len(stocks)):
            for idx_memory in range(self.n_memory):
                action = None
                reward = np.concatenate((np.reshape(stocks[t], (self.n_stock, 1)), np.zeros((self.n_stock, 1))), axis=-1)
                self.memory[idx_memory].append(stocks[t], action, reward)
        
    def update_memory(self, state):
        # update memory without updating weight
        for i in range(self.n_memory):
            self.memory[i].observations.append(state)
            self.memory[i].priority.append(1.0)
        # to stabilize batch normalization, use other samples for prediction
        pred_state = self.memory[0].sample_state_uniform(self.n_batch, self.n_history)
        for i in range(self.n_memory):
            action_off = None
            reward_off = np.concatenate((np.reshape(state, (self.n_stock, 1)), np.zeros((self.n_stock, 1))), axis=-1)
            self.memory[i].rewards.append(reward_off)
            self.memory[i].actions.append(action_off)
    
    def get_max_idx(self, state):
        max_action = self.sess.run(self.max_action_target, feed_dict={self.state_target: state})
        shape = max_action.shape
        max_idx = []
        for i in range(shape[0]):
            for j in range(shape[1]):
                max_idx.append([i, j, max_action[i][j]])
        return np.array(max_idx, dtype=int)
    
    
    def build_model(self):
        """Build all of the network and optimizations
        
        just for conveninece of trainig, seprate placehoder for train and target network
        critic network input: [raw_data, smoothed, downsampled, action]
        actor network input: [raw_data, smoothed, downsampled]
        """
        self.critic = self.build_critic()
        self.critic_target = self.build_critic()
        # transform input into the several scales and smoothing
        self.state =  tf.placeholder(tf.float32, [None, self.n_history, self.n_stock], name='state')
        self.state_target = tf.placeholder(tf.float32, [None, self.n_history, self.n_stock], name='state_target')
        # reshape to convolutional input
        state_ = tf.reshape(self.state, [-1, self.n_history, self.n_stock, 1])
        state_target_ = tf.reshape(self.state_target, [-1, self.n_history, self.n_stock, 1])
        raw, smoothed, down = self.transform_input(state_)
        raw_target, smoothed_target, down_target = self.transform_input(state_target_)
        
        # build graph for citic training
        input_q = [raw,] +  smoothed + down
        self.Q = self.critic(input_q)
        self.max_action = tf.argmax(self.Q, dimension=2)
        # target network
        input_q_target = [raw_target,] +  smoothed_target + down_target
        Q_target = self.critic_target(input_q_target)
        self.reward = tf.placeholder(tf.float32, [None, self.n_stock, 2], name='reward')
        double_Q = self.critic(input_q_target)
        self.max_action_target = tf.argmax(double_Q, 2)
        self.max_idx_target = tf.placeholder(tf.int32, [None, 3], "double_idx")
        Q_max = tf.gather_nd(Q_target, self.max_idx_target)
        Q_max = tf.reshape(Q_max, [-1, self.n_stock, 1])
        Q_value = tf.concat(2, (tf.zeros_like(Q_max), Q_max))
        self.target_value = self.reward  + self.gamma * Q_value
        self.target_value = tf.cast(self.target_value, tf.float32)
        self.target = tf.placeholder(tf.float32, [None, self.n_stock, 2], name="target_value")
        # optimization
        self.learning_rate = tf.placeholder(tf.float32, shape=[], name="learning_rate")
        # get rid of bias of prioritized
        self.weights = tf.placeholder(tf.float32, shape=[None], name="weights")
        self.loss = tf.reduce_mean(self.weights * tf.reduce_sum(tf.square(self.target - self.Q), [1, 2]), name='loss')
        # TD-error for priority
        self.error = tf.reduce_sum(tf.abs(self.target - self.Q), [1, 2])
        self.critic_optim = tf.train.AdamOptimizer(self.learning_rate) \
            .minimize(self.loss, var_list=self.critic.trainable_weights)
        
        self.saver = tf.train.Saver()
        is_initialize = True
        if self.is_load:
            if self.load(self.save_path):
                print('succeded to load')
                is_initialize = False
            else:
                print('failed to load')
        
        # initialize network
        tf.initialize_all_variables().run(session=self.sess)
        weights = self.critic.get_weights()
        self.critic_target.set_weights(weights)
        
    def build_critic(self):
        """Build critic network
        
        recieve convereted tensor: raw_data, smooted_data, and downsampled_data
        """
        nf = self.n_feature
        # layer1
        # smoothed input
        sm_model = [Sequential() for _ in range(self.n_smooth)]
        for m in sm_model:
            m.add(Lambda(lambda x: x,  input_shape=(self.history_length, self.n_stock, 1)))
            # m.add(SpatialDropout2D(0.2))
            m.add(Convolution2D(nb_filter=nf, nb_row=self.k_w, nb_col=1, border_mode='same'))
            m.add(BatchNormalization(mode=2, axis=-1))
            m.add(PReLU())
        # down sampled input
        dw_model = [Sequential() for i in range(self.n_down)]
        for m in dw_model:
            m.add(Lambda(lambda x: x,  input_shape=(self.history_length, self.n_stock, 1)))
            # m.add(SpatialDropout2D(0.2))
            m.add(Convolution2D(nb_filter=nf, nb_row=self.k_w, nb_col=1, border_mode='same'))
            m.add(BatchNormalization(mode=2, axis=-1))
            m.add(PReLU())
        # raw input
        state = Sequential()
        nf = self.n_feature
        state.add(Lambda(lambda x: x,  input_shape=(self.history_length, self.n_stock, 1)))
        # state.add(SpatialDropout2D(0.2))
        state.add(Convolution2D(nb_filter=nf, nb_row=self.k_w, nb_col=1, border_mode='same'))
        state.add(BatchNormalization(mode=2, axis=-1))
        state.add(PReLU())
        merged = Merge([state,] + sm_model + dw_model, mode='concat', concat_axis=-1)
        # layer2
        nf = nf * 2
        model = Sequential()
        model.add(merged)
        # merged_state.add(SpatialDropout2D(0.2))
        model.add(Convolution2D(nb_filter=nf, nb_row=self.k_w, nb_col=1, border_mode='same'))
        model.add(BatchNormalization(mode=2, axis=-1))
        model.add(PReLU())
        model.add(Flatten())
        # layer3
        model.add(Dense(self.n_hidden))
        model.add(BatchNormalization(mode=1, axis=-1))
        model.add(PReLU())
        # layer4
        # model.add(Dropout(0.5))
        model.add(Dense(int(np.sqrt(self.n_hidden))))
        # model.add(BatchNormalization(mode=1, axis=-1))
        model.add(PReLU())
        # output
        # model.add(Dropout(0.5))
        model.add(Dense(2 * self.n_stock))
        model.add(Reshape((self.n_stock, 2)))
        return model
    
    def transform_input(self, input):
        """Transform data into the Multi Scaled one
        
        Args:
            input: tensor with shape: [None, self.n_history, self.n_stock]
        Return:
            list of the same shape tensors, [None, self.length_history, self.n_stock]
        """
        # the last data is the newest information
        raw = input[:, self.n_history - self.history_length:, :, :]
        # smooth data
        smoothed = []
        for n_sm in range(2, self.n_smooth + 2):
            smoothed.append(
                tf.reduce_mean(tf.pack([input[:, self.n_history - st - self.history_length:self.n_history - st, :, :]
                                        for st in range(n_sm)]),0))
        # downsample data
        down = []
        for n_dw in range(2, self.n_down + 2):
            sampled_ = tf.pack([input[:, idx, :, :] 
                                for idx in range(self.n_history-n_dw*self.history_length, self.n_history, n_dw)])
            down.append(tf.transpose(sampled_, [1, 0, 2, 3]))
        return raw, smoothed, down
    
    def load(self, checkpoint_dir):
        print(" [*] Reading checkpoints...")
        try:
            self.saver.restore(self.sess, self.save_path)
            return True
        except:
            return False

In [14]:
# import numpy as np 
# input_data = np.zeros((505, 99))
n_stock = len(input_data.values[0])
# n_stock = 10

class DQNConfig(object):
    device = '/cpu:0'
    save_path = '/home/tomoaki/work/github/DQN/DDPG_model.ckpt'
    is_load = False
    activation = 'relu'
    gamma = 1 - 1.0e-4
    history_length = 10
    n_stock = n_stock
    n_smooth = 5
    n_down = 5
    k_w = 3
    n_hidden = 100
    n_batch = 32
    n_epochs = 100
    n_feature = 32
    alpha = 0.7
    beta = 0.5
    update_rate = 1e-1
    learning_rate = 1e-3
    
    # memory_config
    memory_length = 200
    n_memory = 10
    action_scale = 10

In [17]:
import sys
sys.path.append("./model")
from dqn import DQN
from config import DQNConfig

n_stock = len(train_data.values[0])
config = DQNConfig(n_stock)
dqn = DQN(config)
print ("start!")
values = dqn.train(train_data)
print ("finished!")

9
building model....
finished building model!
start!
training....
Model saved in file: /home/tomoaki/work/github/DQN/DQN_model.ckpt
('time:', Timestamp('2007-04-13 00:00:00'))
('error:', 8.5402746)
('portfolio:', array([1, 1, 1, 1, 1, 1, 1, 1, 1]))
('elapsed time', 233.7516210079193)
********************************************************************
Model saved in file: /home/tomoaki/work/github/DQN/DQN_model.ckpt
Model saved in file: /home/tomoaki/work/github/DQN/DQN_model.ckpt
Model saved in file: /home/tomoaki/work/github/DQN/DQN_model.ckpt
('time:', Timestamp('2007-05-15 00:00:00'))
('error:', 7.0837221)
('portfolio:', array([1, 1, 0, 1, 1, 1, 1, 1, 1]))
('elapsed time', 503.76945090293884)
********************************************************************
Model saved in file: /home/tomoaki/work/github/DQN/DQN_model.ckpt


KeyboardInterrupt: 

In [15]:
# from model import DDPG
import warnings
warnings.filterwarnings("ignore")

config = DQNConfig()

dqn = DQN(config)
print ("start!")
values = dqn.train(train_data)
print ("finished!")

building model....
finished building model!
start!
training....


KeyboardInterrupt: 

In [ ]:
i = 9

plt.scatter(values[i].index, values[i].values)
plt.plot(input_tilde.index, input_tilde.values[:, i])